# Sway hull equation

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sympy as sp
from sympy.plotting import plot as plot
from sympy.plotting import plot3d as plot3d
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sp.init_printing()
from IPython.core.display import HTML

In [ ]:
import seaman.helpers
import seaman_symbol as ss
import sway_hull_equations as equations
import sway_hull_lambda_functions as lambda_functions
from bis_system import BisSystem

## Coordinate system
![coordinate_system](coordinate_system.png)

## Symbols

In [ ]:
from seaman_symbols import *

In [ ]:
HTML(ss.create_html_table(symbols=equations.total_sway_hull_equation_SI.free_symbols))

## Sway equation

In [ ]:
equations.sway_hull_equation

### Force due to drift

In [ ]:
equations.sway_drift_equation

Same equation in SI units

In [ ]:
equations.sway_drift_equation_SI

### Force due to yaw rate

In [ ]:
equations.sway_yaw_rate_equation

In [ ]:
equations.sway_yaw_rate_equation_SI

### Nonlinear force
The nonlinear force is calculated as the sectional cross flow drag.

![cd](cd.png)

In [ ]:
equations.sway_none_linear_equation

Simple assumption for section draught:

In [ ]:
equations.section_draught_equation

In [ ]:
equations.simplified_sway_none_linear_equation

Nonlinear force equation expressed as bis force:

In [ ]:
equations.simplified_sway_none_linear_equation_bis

In [ ]:
equations.sway_hull_equation_SI

In [ ]:
equations.sway_hull_equation_SI

In [ ]:
equations.total_sway_hull_equation_SI

### Plotting the total sway hull force equation

In [ ]:
df = pd.DataFrame()
df['v_w'] = np.linspace(-0.3,3,10)
df['u_w'] = 5.0
df['r_w'] = 0.0
df['rho'] = 1025
df['t_a'] = 1.0
df['t_f'] = 1.0
df['L'] = 1.0
df['Y_uv'] = 1.0
df['Y_uuv'] = 1.0
df['Y_ur'] = 1.0
df['Y_uur'] = 1.0
df['C_d'] = 0.5
df['g'] = 9.81
df['disp'] = 23

result = df.copy()
result['fy'] = lambda_functions.Y_h_function(**df)

result.plot(x = 'v_w',y = 'fy');

### Plotting with coefficients from a real seaman ship model

In [ ]:
import generate_input
shipdict = seaman.ShipDict.load('../../tests/test_ship.ship')

In [ ]:
df = pd.DataFrame()
df['v_w'] = np.linspace(-3,3,20)
df['rho'] = 1025.0
df['g'] = 9.81
df['u_w'] = 5.0
df['r_w'] = 0.0

df_input = generate_input.add_shipdict_inputs(lambda_function=lambda_functions.Y_h_function,
                                              shipdict = shipdict,
                                              df = df)
df_input

In [ ]:
result = df_input.copy()
result['fy'] = lambda_functions.Y_h_function(**df_input)

In [ ]:
result.plot(x = 'v_w',y = 'fy');

## Real seaman++
Run real seaman in C++ to verify that the documented model is correct. 

In [ ]:
import run_real_seaman

In [ ]:
df = pd.DataFrame()
df['v_w'] = np.linspace(-3,3,20)
df['rho'] = 1025.0
df['g'] = 9.81
df['u_w'] = 5.0
df['r_w'] = 0.0

result_comparison = run_real_seaman.compare_with_seaman(lambda_function=lambda_functions.Y_h_function,
                                                        shipdict = shipdict,
                                                        df = df)

fig,ax = plt.subplots()
result_comparison.plot(x = 'v_w',y = ['fy','fy_seaman'],ax = ax)
ax.set_title('Drift angle variation');

In [ ]:
df = pd.DataFrame()
df['r_w'] = np.linspace(-0.1,0.1,20)
df['rho'] = 1025.0
df['g'] = 9.81
df['u_w'] = 5.0
df['v_w'] = 0.0


df_input = generate_input.add_shipdict_inputs(lambda_function=lambda_functions.Y_h_function,
                                              shipdict = shipdict,
                                              df = df)

result_comparison = run_real_seaman.compare_with_seaman(lambda_function=lambda_functions.Y_h_function,
                                                        shipdict = shipdict,
                                                        df = df,)
fig,ax = plt.subplots()
result_comparison.plot(x = 'r_w',y = ['fy','fy_seaman'],ax = ax)
ax.set_title('Yaw rate variation');

In [ ]:
df_input